In [ ]:
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import cartopy
import pickle
import xarray as xa
from matplotlib import colors
from matplotlib.gridspec import GridSpec, GridSpecFromSubplotSpec
import matplotlib as mpl
from scipy.stats import pearsonr
import xcdat as xc
from sklearn.linear_model import LinearRegression

In [ ]:
mpl.rc('font', family='DejaVu Serif') 
mpl.rcParams['mathtext.rm']='DejaVu Serif'
mpl.rcParams['mathtext.fontset'] = 'custom'
# mpl.rc('font', serif='Helvetica Neue') 

In [ ]:
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-False-joint-False', 'rb') as pfile:
    solver_list_month_stand = pickle.load(pfile)

In [ ]:
len(solver_list_month_stand)

In [ ]:
# get patterns:
neofs=0
eofs_all = []
for i in range(12):
    eof = solver_list_month_stand[i].eofs().isel(mode=neofs).transpose('lat', 'lon')
    pc = solver_list_month_stand[i].pcs().isel(mode=neofs)
    m, b = np.polyfit(np.arange(len(pc)), pc, deg=1)
    if m<0:
        print(i)
        eof = -eof
    eofs_all.append(eof)

In [ ]:
for i in range(12):
    eofs_all[i].mean(dim='lon').plot()
plt.show()

In [ ]:
variable = 'pr'
eof_start = 1979
with open('/p/lustre2/shiduan/ForceSMIP/EOF/modes_all/'+str(eof_start)+'_2022/'+variable+'-solver-stand-True-month-True-unforced-False-joint-False-REGEN-mask', 'rb') as pfile:
    solver_list_month_stand = pickle.load(pfile)

In [ ]:
# get patterns:
neofs=0
eofs_all_land = []
for i in range(12):
    eof = solver_list_month_stand[i].eofs().isel(mode=neofs).transpose('lat', 'lon')
    pc = solver_list_month_stand[i].pcs().isel(mode=neofs)
    m, b = np.polyfit(np.arange(len(pc)), pc, deg=1)
    if m<0:
        print(i)
        eof = -eof
    eofs_all_land.append(eof)

In [ ]:
for i in range(12):
    eofs_all_land[i].mean(dim='lon').plot()
plt.show()

In [ ]:
for i in [0, 3, 6, 9]:
    eofs_all[i].mean(dim='lon').plot(label=str(i+1))
plt.legend()
plt.title('global')
plt.show()
for i in [0, 3, 6, 9]:
    eofs_all_land[i].mean(dim='lon').plot(label=str(i+1))
plt.legend()
plt.title('land')
plt.show()

In [ ]:
gpcp = []
for i in range(1983, 2021):
    data = xc.open_dataset('/p/lustre3/shiduan/GPCP/regrid/'+str(i)+'.nc')
    gpcp.append(data)
gpcp = xa.concat(gpcp, dim='time')
gpcp["__xarray_dataarray_variable__"] = gpcp["__xarray_dataarray_variable__"].transpose('time', 'lon', 'lat')
gpcp = gpcp.bounds.add_missing_bounds()
gpcp_climate = gpcp.temporal.climatology('__xarray_dataarray_variable__', freq='month')
gpcp_zonal_climate = gpcp.mean(dim='lon').temporal.climatology('__xarray_dataarray_variable__', freq='month')

In [ ]:
mswep = xc.open_dataset(
    '/p/lustre3/shiduan/MSWEP/MSWEP-V280-Past-v20231102-monpr-forcesmip.nc')
mswep = mswep.fillna(0)
mswep = mswep.sel(time=slice('1983-01-01', '2021-01-01'))
mswep["__xarray_dataarray_variable__"] = mswep["__xarray_dataarray_variable__"].transpose('time', 'lon', 'lat')
mswep = mswep.bounds.add_missing_bounds()
mswep_climate = mswep.temporal.climatology('__xarray_dataarray_variable__', freq='month')
mswep_zonal_climate = mswep.mean(dim='lon').temporal.climatology('__xarray_dataarray_variable__', freq='month')

In [ ]:
maskfile = "/p/lustre1/shiduan/REGEN/REGEN_mask_forcesmip.nc"
missing_data_maskx = xa.open_dataset(maskfile)
missing_data = np.where(np.isnan(missing_data_maskx.p.transpose('lon', 'lat')), np.nan, 1)
missing_xa = xa.where(np.isnan(missing_data_maskx.p), np.nan, 1)

In [ ]:
mswep_land = mswep['__xarray_dataarray_variable__']*missing_xa
gpcp_land = gpcp['__xarray_dataarray_variable__']*missing_xa
gpcp_land = gpcp_land.to_dataset(name='precipitation')
mswep_land = mswep_land.to_dataset(name='precipitation')
gpcp_land = gpcp_land.bounds.add_missing_bounds()
mswep_land = mswep_land.bounds.add_missing_bounds()


In [ ]:
mswep_zonal_climate_land = mswep_land.mean(dim='lon').temporal.climatology('precipitation', freq='month')
gpcp_zonal_climate_land = gpcp_land.mean(dim='lon').temporal.climatology('precipitation', freq='month')

# Marvel curve

In [ ]:
gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=0).plot()
gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=3).plot()
gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=6).plot()
gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=9).plot()
plt.show()
gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=0).rolling(lat=3, center=True).mean().plot()
gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=3).rolling(lat=3, center=True).mean().plot()
gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=6).rolling(lat=3, center=True).mean().plot()
gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=9).rolling(lat=3, center=True).mean().plot()
plt.show()


In [ ]:
fig = plt.figure()
ax = fig.add_subplot(221)
(gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=0).rolling(lat=5, center=True).mean()/600*2-0.01).plot(ax=ax, color='green')
(mswep_zonal_climate['__xarray_dataarray_variable__'].isel(time=0).rolling(lat=5, center=True).mean()/600*2-0.01).plot(ax=ax, color='green')
eofs_all[0].mean(dim='lon').plot(ax=ax, color='black')
ax.axhline(0, color='black')
ax = fig.add_subplot(222)
(gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=3).rolling(lat=5, center=True).mean()/600*2-0.01).plot(ax=ax, color='green')
(mswep_zonal_climate['__xarray_dataarray_variable__'].isel(time=3).rolling(lat=5, center=True).mean()/600*2-0.01).plot(ax=ax, color='green')
eofs_all[3].mean(dim='lon').plot(ax=ax, color='black')
ax.axhline(0, color='black')
ax = fig.add_subplot(223)
(gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=6).rolling(lat=5, center=True).mean()/600*2-0.01).plot(ax=ax, color='green')
(mswep_zonal_climate['__xarray_dataarray_variable__'].isel(time=6).rolling(lat=5, center=True).mean()/600*2-0.01).plot(ax=ax, color='green')
eofs_all[6].mean(dim='lon').plot(ax=ax, color='black')
ax.axhline(0, color='black')
ax = fig.add_subplot(224)
(gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=9).rolling(lat=5, center=True).mean()/600*2-0.01).plot(ax=ax, color='green')
(mswep_zonal_climate['__xarray_dataarray_variable__'].isel(time=9).rolling(lat=5, center=True).mean()/600*2-0.01).plot(ax=ax, color='green')
eofs_all[9].mean(dim='lon').plot(ax=ax, color='black')
ax.axhline(0, color='black')
plt.tight_layout()
plt.show()

In [ ]:
from scipy.signal import argrelextrema
p = gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(time=0).rolling(lat=5, center=True).mean()
# p.plot()
print(argrelextrema(p.data, np.greater)[0])
print(argrelextrema(p.data, np.less)[0])
print(p.lat[16].data, p.lat[32].data, p.lat[51].data)
print(p.lat[23].data, p.lat[43].data)

In [ ]:
def find_closest_lat(p, target_value):
    closest_p = p.where(np.abs(p - target_value) == np.abs(p - target_value).min(), drop=True)
    return closest_p.lat.data[0]

In [ ]:
fig = plt.figure(figsize=(20, 5))
for i in range(12):
    ax = fig.add_subplot(2, 6, i+1)
    p = gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(
        time=i).rolling(lat=5, center=True).mean()
    p = p.dropna(dim='lat')
    min_lat = p.lat.data[0]
    max_lat = p.lat.data[-1]
    wet_indices = argrelextrema(p.data, np.greater)[0]
    e_lat = eofs_all[i].mean(dim='lon').sel(lat=slice(min_lat, max_lat))
    p.plot(ax=ax, linewidth=2, color='tab:blue')
    organized_wet = []
    wet_lat = p.lat[wet_indices].data
    local_maxima_latitudes_sorted = sorted(wet_lat)  # Sort by latitude
    p6 = min(local_maxima_latitudes_sorted, key=lambda x: abs(x))
    organized_wet.append(p6)
    p2 = min([lat for lat in local_maxima_latitudes_sorted if lat < p6], default=None)
    p10 = max([lat for lat in local_maxima_latitudes_sorted if lat > p6], default=None)
    organized_wet.append(p2)
    organized_wet.append(p10)
    organized_dry = []
    left_precip = p.sel(lat=slice(p2, p6))
    p4 = left_precip.lat[left_precip.argmin(dim='lat').data].data
    organized_dry.append(p4)
    right_precip = p.sel(lat=slice(p6, p10))
    p8 = right_precip.lat[right_precip.argmin(dim='lat').data].data
    organized_dry.append(p8)
    p1 = find_closest_lat(p.sel(lat=slice(-90, p2)), p.sel(lat=p2).data / 2)
    p3 = find_closest_lat(p.sel(lat=slice(p2, p4)), (p.sel(lat=p2).data + p.sel(lat=p4).data) / 2)
    p5 = find_closest_lat(p.sel(lat=slice(p4, p6)), (p.sel(lat=p4).data + p.sel(lat=p6).data) / 2)
    p7 = find_closest_lat(p.sel(lat=slice(p6, p8)), (p.sel(lat=p6).data + p.sel(lat=p8).data) / 2)
    p9 = find_closest_lat(p.sel(lat=slice(p8, p10)), (p.sel(lat=p8).data + p.sel(lat=p10).data) / 2)
    p11 = find_closest_lat(p.sel(lat=slice(p10, 90)), p.sel(lat=p10).data / 2)
    '''for d in organized_wet:
        ax.axvline(d, linestyle='--')
    for d in organized_dry:
        ax.axvline(d, linestyle='--')'''
    ax.set_title('Month: '+str(i+1))
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.axvspan(p5, p7, alpha=0.15, color='tab:green')
    ax.axvspan(p7, p9, alpha=0.15, color='tab:brown')
    ax.axvspan(p3, p5, alpha=0.15, color='tab:brown')
    ax.axvspan(p9, max_lat, alpha=0.15, color='tab:green')
    ax.axvspan(min_lat, p3, alpha=0.15, color='tab:green')
    ax.set_yticks([0, 2, 4, 6])
    ax.set_ylim([0, 6.5])
    ax = ax.twinx() # eof plot
    e_lat.plot(ax=ax, color='black', alpha=.5, linewidth=2)
    # MidLatitudes
    right_eof = e_lat.sel(lat=slice(p9, max_lat))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:green', interpolate=True)
    left_eof = e_lat.sel(lat=slice(min_lat, p3))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:green', interpolate=True)
    if left_eof.mean().data>0: # movement
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:cyan', interpolate=True)
    if right_eof.mean().data>0:
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:cyan', interpolate=True)
    # subtropics
    right_eof = e_lat.sel(lat=slice(p7, p9))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:red', interpolate=True)
    if right_eof.mean().data<0:
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:cyan', interpolate=True)
    left_eof = e_lat.sel(lat=slice(p3, p5))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:red', interpolate=True)
    if left_eof.mean().data<0:
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:cyan', interpolate=True)
    # tropics
    tropic_eof = e_lat.sel(lat=slice(p5, p7))
    ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof>=0, color='tab:green', interpolate=True)
    if tropic_eof.mean().data>0:
        ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof<=0, color='tab:cyan', interpolate=True)
    ax.set_title('')
    ax.set_ylabel('')
    ax.set_ylim([-0.02, 0.02])
    ax.axhline(0, color='black', alpha=.5)
plt.tight_layout()
plt.show()

In [ ]:
p1

In [ ]:
gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(
        time=5).rolling(lat=5, center=True).mean().plot()
gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(
        time=5).rolling(lat=7, center=True).mean().plot()

In [ ]:
fig = plt.figure(figsize=(20, 5))
for i in range(12):
    ax = fig.add_subplot(2, 6, i+1)
    p = gpcp_zonal_climate_land['precipitation'].isel(
        time=i).rolling(lat=5, center=True).mean()
    p = p.dropna(dim='lat')
    min_lat = p.lat.data[0]
    max_lat = p.lat.data[-1]
    wet_indices = argrelextrema(p.data, np.greater)[0]
    e_lat = eofs_all_land[i].mean(dim='lon').sel(lat=slice(min_lat, max_lat))
    p.plot(ax=ax, linewidth=2, color='tab:blue')
    organized_wet = []
    wet_lat = p.lat[wet_indices].data
    local_maxima_latitudes_sorted = sorted(wet_lat)  # Sort by latitude
    p6 = min(local_maxima_latitudes_sorted, key=lambda x: abs(x))
    organized_wet.append(p6)
    p2 = min([lat for lat in local_maxima_latitudes_sorted if lat < p6], default=min_lat)
    p10 = max([lat for lat in local_maxima_latitudes_sorted if lat > p6], default=max_lat)
    organized_wet.append(p2)
    organized_wet.append(p10)
    organized_dry = []
    left_precip = p.sel(lat=slice(p2, p6))
    p4 = left_precip.lat[left_precip.argmin(dim='lat').data].data
    organized_dry.append(p4)
    right_precip = p.sel(lat=slice(p6, p10))
    p8 = right_precip.lat[right_precip.argmin(dim='lat').data].data
    organized_dry.append(p8)
    p1 = find_closest_lat(p.sel(lat=slice(min_lat, p2)), p.sel(lat=p2).data / 2)
    p3 = find_closest_lat(p.sel(lat=slice(p2, p4)), (p.sel(lat=p2).data + p.sel(lat=p4).data) / 2)
    p5 = find_closest_lat(p.sel(lat=slice(p4, p6)), (p.sel(lat=p4).data + p.sel(lat=p6).data) / 2)
    p7 = find_closest_lat(p.sel(lat=slice(p6, p8)), (p.sel(lat=p6).data + p.sel(lat=p8).data) / 2)
    p9 = find_closest_lat(p.sel(lat=slice(p8, p10)), (p.sel(lat=p8).data + p.sel(lat=p10).data) / 2)
    p11 = find_closest_lat(p.sel(lat=slice(p10, max_lat)), p.sel(lat=p10).data / 2)
    '''for d in organized_wet:
        ax.axvline(d, linestyle='--')
    for d in organized_dry:
        ax.axvline(d, linestyle='--')'''
    ax.set_title('Month: '+str(i+1))
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.axvspan(p5, p7, alpha=0.15, color='tab:green')
    ax.axvspan(p7, p9, alpha=0.15, color='tab:brown')
    ax.axvspan(p3, p5, alpha=0.15, color='tab:brown')
    ax.axvspan(p9, max_lat, alpha=0.15, color='tab:green')
    ax.axvspan(min_lat, p3, alpha=0.15, color='tab:green')
    ax.set_yticks([0, 4, 8])
    ax.set_ylim([0, 8.5])
    ax = ax.twinx() # eof plot
    e_lat.plot(ax=ax, color='black', alpha=.5, linewidth=2)
    # MidLatitudes
    right_eof = e_lat.sel(lat=slice(p9, max_lat))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:green', interpolate=True)
    left_eof = e_lat.sel(lat=slice(min_lat, p3))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:green', interpolate=True)
    if left_eof.mean().data>0: # movement
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:cyan', interpolate=True)
    if right_eof.mean().data>0:
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:cyan', interpolate=True)
    # subtropics
    right_eof = e_lat.sel(lat=slice(p7, p9))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:red', interpolate=True)
    if right_eof.mean().data<0:
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:cyan', interpolate=True)
    left_eof = e_lat.sel(lat=slice(p3, p5))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:red', interpolate=True)
    if left_eof.mean().data<0:
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:cyan', interpolate=True)
    # tropics
    tropic_eof = e_lat.sel(lat=slice(p5, p7))
    ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof>=0, color='tab:green', interpolate=True)
    if tropic_eof.mean().data>0:
        ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof<=0, color='tab:cyan', interpolate=True)
    ax.set_title('')
    ax.set_ylabel('')
    ax.set_ylim([-0.05, 0.05])
    ax.set_yticks([-0.04, 0, 0.04])
    ax.axhline(0, color='black', alpha=.5)
plt.tight_layout()
plt.show()

In [ ]:
print(p2)

# Combine land and global

In [ ]:
fig = plt.figure(figsize=(10, 4))
for ind, i in enumerate([0, 3, 6, 9]):
    ax = fig.add_subplot(2, 4, ind+1)
    if ind==0:
        ax.annotate('a)', xy=(-0.15, 1.08), xycoords='axes fraction', weight='bold', fontsize=12)
    p = gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(
        time=i).rolling(lat=5, center=True).mean()
    p = p.dropna(dim='lat')
    min_lat = p.lat.data[0]
    max_lat = p.lat.data[-1]
    wet_indices = argrelextrema(p.data, np.greater)[0]
    e_lat = eofs_all[i].mean(dim='lon').sel(lat=slice(min_lat, max_lat))
    p.plot(ax=ax, linewidth=2, color='tab:blue')
    organized_wet = []
    wet_lat = p.lat[wet_indices].data
    local_maxima_latitudes_sorted = sorted(wet_lat)  # Sort by latitude
    p6 = min(local_maxima_latitudes_sorted, key=lambda x: abs(x))
    organized_wet.append(p6)
    p2 = min([lat for lat in local_maxima_latitudes_sorted if lat < p6], default=None)
    p10 = max([lat for lat in local_maxima_latitudes_sorted if lat > p6], default=None)
    organized_wet.append(p2)
    organized_wet.append(p10)
    organized_dry = []
    left_precip = p.sel(lat=slice(p2, p6))
    p4 = left_precip.lat[left_precip.argmin(dim='lat').data].data
    organized_dry.append(p4)
    right_precip = p.sel(lat=slice(p6, p10))
    p8 = right_precip.lat[right_precip.argmin(dim='lat').data].data
    organized_dry.append(p8)
    p1 = find_closest_lat(p.sel(lat=slice(-90, p2)), p.sel(lat=p2).data / 2)
    p3 = find_closest_lat(p.sel(lat=slice(p2, p4)), (p.sel(lat=p2).data + p.sel(lat=p4).data) / 2)
    p5 = find_closest_lat(p.sel(lat=slice(p4, p6)), (p.sel(lat=p4).data + p.sel(lat=p6).data) / 2)
    p7 = find_closest_lat(p.sel(lat=slice(p6, p8)), (p.sel(lat=p6).data + p.sel(lat=p8).data) / 2)
    p9 = find_closest_lat(p.sel(lat=slice(p8, p10)), (p.sel(lat=p8).data + p.sel(lat=p10).data) / 2)
    p11 = find_closest_lat(p.sel(lat=slice(p10, 90)), p.sel(lat=p10).data / 2)
    ax.set_title('Month: '+str(i+1))
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.axvspan(p5, p7, alpha=0.15, color='tab:green')
    ax.axvspan(p7, p9, alpha=0.15, color='tab:brown')
    ax.axvspan(p3, p5, alpha=0.15, color='tab:brown')
    ax.axvspan(p9, max_lat, alpha=0.15, color='tab:green')
    ax.axvspan(min_lat, p3, alpha=0.15, color='tab:green')
    ax.set_yticks([0, 2, 4, 6])
    ax.set_ylim([0, 6.5])
    ax = ax.twinx() # eof plot
    e_lat.plot(ax=ax, color='black', alpha=.5, linewidth=2)
    # MidLatitudes
    right_eof = e_lat.sel(lat=slice(p9, max_lat))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:green', interpolate=True)
    left_eof = e_lat.sel(lat=slice(min_lat, p3))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:green', interpolate=True)
    if left_eof.mean().data>0: # movement
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:cyan', interpolate=True)
    if right_eof.mean().data>0:
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:cyan', interpolate=True)
    # subtropics
    right_eof = e_lat.sel(lat=slice(p7, p9))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:red', interpolate=True)
    if right_eof.mean().data<0:
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:cyan', interpolate=True)
    left_eof = e_lat.sel(lat=slice(p3, p5))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:red', interpolate=True)
    if left_eof.mean().data<0:
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:cyan', interpolate=True)
    # tropics
    tropic_eof = e_lat.sel(lat=slice(p5, p7))
    ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof>=0, color='tab:green', interpolate=True)
    if tropic_eof.mean().data>0:
        ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof<=0, color='tab:cyan', interpolate=True)
    ax.set_title('')
    ax.set_ylabel('')
    ax.set_ylim([-0.02, 0.02])
    ax.set_yticks([-0.02, 0, 0.02])
    ax.axhline(0, color='black', alpha=.5)
for ind, i in enumerate([0, 3, 6, 9]):
    ax = fig.add_subplot(2, 4, ind+1+4)
    if ind==0:
        ax.annotate('b)', xy=(-0.15, 1.08), xycoords='axes fraction', weight='bold', fontsize=12)
    p = gpcp_zonal_climate_land['precipitation'].isel(
        time=i).rolling(lat=5, center=True).mean()
    p = p.dropna(dim='lat')
    min_lat = p.lat.data[0]
    max_lat = p.lat.data[-1]
    wet_indices = argrelextrema(p.data, np.greater)[0]
    e_lat = eofs_all_land[i].mean(dim='lon').sel(lat=slice(min_lat, max_lat))
    p.plot(ax=ax, linewidth=2, color='tab:blue')
    organized_wet = []
    wet_lat = p.lat[wet_indices].data
    local_maxima_latitudes_sorted = sorted(wet_lat)  # Sort by latitude
    p6 = min(local_maxima_latitudes_sorted, key=lambda x: abs(x))
    organized_wet.append(p6)
    p2 = min([lat for lat in local_maxima_latitudes_sorted if lat < p6], default=min_lat)
    p10 = max([lat for lat in local_maxima_latitudes_sorted if lat > p6], default=max_lat)
    organized_wet.append(p2)
    organized_wet.append(p10)
    organized_dry = []
    left_precip = p.sel(lat=slice(p2, p6))
    p4 = left_precip.lat[left_precip.argmin(dim='lat').data].data
    organized_dry.append(p4)
    right_precip = p.sel(lat=slice(p6, p10))
    p8 = right_precip.lat[right_precip.argmin(dim='lat').data].data
    organized_dry.append(p8)
    p1 = find_closest_lat(p.sel(lat=slice(min_lat, p2)), p.sel(lat=p2).data / 2)
    p3 = find_closest_lat(p.sel(lat=slice(p2, p4)), (p.sel(lat=p2).data + p.sel(lat=p4).data) / 2)
    p5 = find_closest_lat(p.sel(lat=slice(p4, p6)), (p.sel(lat=p4).data + p.sel(lat=p6).data) / 2)
    p7 = find_closest_lat(p.sel(lat=slice(p6, p8)), (p.sel(lat=p6).data + p.sel(lat=p8).data) / 2)
    p9 = find_closest_lat(p.sel(lat=slice(p8, p10)), (p.sel(lat=p8).data + p.sel(lat=p10).data) / 2)
    p11 = find_closest_lat(p.sel(lat=slice(p10, max_lat)), p.sel(lat=p10).data / 2)
    ax.set_title('Month: '+str(i+1))
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.axvspan(p5, p7, alpha=0.15, color='tab:green')
    ax.axvspan(p7, p9, alpha=0.15, color='tab:brown')
    ax.axvspan(p3, p5, alpha=0.15, color='tab:brown')
    ax.axvspan(p9, max_lat, alpha=0.15, color='tab:green')
    ax.axvspan(min_lat, p3, alpha=0.15, color='tab:green')
    ax.set_yticks([0, 4, 8])
    ax.set_ylim([0, 8.5])
    ax = ax.twinx() # eof plot
    e_lat.plot(ax=ax, color='black', alpha=.5, linewidth=2)
    # MidLatitudes
    right_eof = e_lat.sel(lat=slice(p9, max_lat))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:green', interpolate=True)
    left_eof = e_lat.sel(lat=slice(min_lat, p3))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:green', interpolate=True)
    if left_eof.mean().data>0: # movement
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:cyan', interpolate=True)
    if right_eof.mean().data>0:
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:cyan', interpolate=True)
    # subtropics
    right_eof = e_lat.sel(lat=slice(p7, p9))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:red', interpolate=True)
    if right_eof.mean().data<0:
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:cyan', interpolate=True)
    left_eof = e_lat.sel(lat=slice(p3, p5))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:red', interpolate=True)
    if left_eof.mean().data<0:
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:cyan', interpolate=True)
    # tropics
    tropic_eof = e_lat.sel(lat=slice(p5, p7))
    ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof>=0, color='tab:green', interpolate=True)
    if tropic_eof.mean().data>0:
        ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof<=0, color='tab:cyan', interpolate=True)
    ax.set_title('')
    ax.set_ylabel('')
    ax.set_ylim([-0.05, 0.05])
    ax.set_yticks([-0.04, 0, 0.04])
    ax.axhline(0, color='black', alpha=.5)
plt.tight_layout()
plt.savefig('DDWW.png', dpi=180, bbox_inches='tight')
plt.show()

In [ ]:
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
fig, axes = plt.subplots(figsize=(14, 7), ncols=6, nrows=4, sharex='row', sharey='row')
for ind, i in enumerate(range(12)):
    # ax = fig.add_subplot(4, 6, ind+1)
    ax = axes.flatten()[ind]
    if ind==0:
        ax.annotate('a)', xy=(-0.15, 1.08), xycoords='axes fraction', weight='bold', fontsize=12)
    p = mswep_zonal_climate['__xarray_dataarray_variable__'].isel(
        time=i).rolling(lat=5, center=True).mean()
    p = p.dropna(dim='lat')
    min_lat = p.lat.data[0]
    max_lat = p.lat.data[-1]
    wet_indices = argrelextrema(p.data, np.greater)[0]
    e_lat = eofs_all[i].mean(dim='lon').sel(lat=slice(min_lat, max_lat))
    p.plot(ax=ax, linewidth=2, color='tab:blue')
    organized_wet = []
    wet_lat = p.lat[wet_indices].data
    local_maxima_latitudes_sorted = sorted(wet_lat)  # Sort by latitude
    p6 = min(local_maxima_latitudes_sorted, key=lambda x: abs(x))
    organized_wet.append(p6)
    p2 = min([lat for lat in local_maxima_latitudes_sorted if lat < p6], default=None)
    p10 = max([lat for lat in local_maxima_latitudes_sorted if lat > p6], default=None)
    organized_wet.append(p2)
    organized_wet.append(p10)
    organized_dry = []
    left_precip = p.sel(lat=slice(p2, p6))
    p4 = left_precip.lat[left_precip.argmin(dim='lat').data].data
    organized_dry.append(p4)
    right_precip = p.sel(lat=slice(p6, p10))
    p8 = right_precip.lat[right_precip.argmin(dim='lat').data].data
    organized_dry.append(p8)
    p1 = find_closest_lat(p.sel(lat=slice(-90, p2)), p.sel(lat=p2).data / 2)
    p3 = find_closest_lat(p.sel(lat=slice(p2, p4)), (p.sel(lat=p2).data + p.sel(lat=p4).data) / 2)
    p5 = find_closest_lat(p.sel(lat=slice(p4, p6)), (p.sel(lat=p4).data + p.sel(lat=p6).data) / 2)
    p7 = find_closest_lat(p.sel(lat=slice(p6, p8)), (p.sel(lat=p6).data + p.sel(lat=p8).data) / 2)
    p9 = find_closest_lat(p.sel(lat=slice(p8, p10)), (p.sel(lat=p8).data + p.sel(lat=p10).data) / 2)
    p11 = find_closest_lat(p.sel(lat=slice(p10, 90)), p.sel(lat=p10).data / 2)
    ax.set_title(month_names[i])
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.axvspan(p5, p7, alpha=0.15, color='tab:green')
    ax.axvspan(p7, p9, alpha=0.15, color='tab:brown')
    ax.axvspan(p3, p5, alpha=0.15, color='tab:brown')
    ax.axvspan(p9, max_lat, alpha=0.15, color='tab:green')
    ax.axvspan(min_lat, p3, alpha=0.15, color='tab:green')
    ax.set_yticks([0, 2, 4, 6])
    ax.set_ylim([0, 6.5])
    ax = ax.twinx() # eof plot
    e_lat.plot(ax=ax, color='black', alpha=.5, linewidth=2)
    # MidLatitudes
    right_eof = e_lat.sel(lat=slice(p9, max_lat))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:green', interpolate=True) # wetter
    left_eof = e_lat.sel(lat=slice(min_lat, p3))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:green', interpolate=True) # wetter
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:orange', interpolate=True) # wet get drier
    if left_eof.mean().data>0: # movement
        pass
    else: # wet get drier
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, facecolor='none', interpolate=True, hatch="xxxx")
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:orange', interpolate=True) # wet get drier
    if right_eof.mean().data>0:
        pass
    else: # wet get drier
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, facecolor='none', interpolate=True, hatch="xxxx")
    # subtropics
    right_eof = e_lat.sel(lat=slice(p7, p9))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:red', interpolate=True) # drier
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:cyan', interpolate=True) # dry get wetter
    if right_eof.mean().data<0: # movement
        pass
    else:
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, facecolor='none', interpolate=True, hatch="xxxx") # dry get wetter
    left_eof = e_lat.sel(lat=slice(p3, p5))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:red', interpolate=True)
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:cyan', interpolate=True)
    if left_eof.mean().data<0:
        pass
    else:
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, facecolor='none', interpolate=True, hatch="xxxx")
    # tropics
    tropic_eof = e_lat.sel(lat=slice(p5, p7))
    ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof>=0, color='tab:green', interpolate=True)
    ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof<=0, color='tab:orange', interpolate=True) # wet get drier
    if tropic_eof.mean().data>0:
        pass
    else:
        ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof<=0, facecolor='none', interpolate=True, hatch='xxxx')
    ax.set_title('')
    ax.set_ylabel('')
    ax.set_ylim([-0.02, 0.02])
    ax.set_yticks([-0.02, 0, 0.02])
    if ind==11 or ind==5:
        pass
    else:
        ax.tick_params(labelright=False)
    ax.axhline(0, color='black', alpha=.5)
for ind, i in enumerate(range(12)):
    # ax = fig.add_subplot(4, 6, ind+1+12)
    ax = axes.flatten()[ind+12]
    if ind==0:
        ax.annotate('b)', xy=(-0.15, 1.08), xycoords='axes fraction', weight='bold', fontsize=12)
    p = mswep_zonal_climate_land['precipitation'].isel(
        time=i).rolling(lat=5, center=True).mean()
    p = p.dropna(dim='lat')
    min_lat = p.lat.data[0]
    max_lat = p.lat.data[-1]
    wet_indices = argrelextrema(p.data, np.greater)[0]
    e_lat = eofs_all_land[i].mean(dim='lon').sel(lat=slice(min_lat, max_lat))
    p.plot(ax=ax, linewidth=2, color='tab:blue')
    organized_wet = []
    wet_lat = p.lat[wet_indices].data
    local_maxima_latitudes_sorted = sorted(wet_lat)  # Sort by latitude
    p6 = min(local_maxima_latitudes_sorted, key=lambda x: abs(x))
    organized_wet.append(p6)
    p2 = min([lat for lat in local_maxima_latitudes_sorted if lat < p6], default=min_lat)
    p10 = max([lat for lat in local_maxima_latitudes_sorted if lat > p6], default=max_lat)
    organized_wet.append(p2)
    organized_wet.append(p10)
    organized_dry = []
    left_precip = p.sel(lat=slice(p2, p6))
    p4 = left_precip.lat[left_precip.argmin(dim='lat').data].data
    organized_dry.append(p4)
    right_precip = p.sel(lat=slice(p6, p10))
    p8 = right_precip.lat[right_precip.argmin(dim='lat').data].data
    organized_dry.append(p8)
    p1 = find_closest_lat(p.sel(lat=slice(min_lat, p2)), p.sel(lat=p2).data / 2)
    p3 = find_closest_lat(p.sel(lat=slice(p2, p4)), (p.sel(lat=p2).data + p.sel(lat=p4).data) / 2)
    p5 = find_closest_lat(p.sel(lat=slice(p4, p6)), (p.sel(lat=p4).data + p.sel(lat=p6).data) / 2)
    p7 = find_closest_lat(p.sel(lat=slice(p6, p8)), (p.sel(lat=p6).data + p.sel(lat=p8).data) / 2)
    p9 = find_closest_lat(p.sel(lat=slice(p8, p10)), (p.sel(lat=p8).data + p.sel(lat=p10).data) / 2)
    p11 = find_closest_lat(p.sel(lat=slice(p10, max_lat)), p.sel(lat=p10).data / 2)
    ax.set_title(month_names[i])
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.axvspan(p5, p7, alpha=0.15, color='tab:green')
    ax.axvspan(p7, p9, alpha=0.15, color='tab:brown')
    ax.axvspan(p3, p5, alpha=0.15, color='tab:brown')
    ax.axvspan(p9, max_lat, alpha=0.15, color='tab:green')
    ax.axvspan(min_lat, p3, alpha=0.15, color='tab:green')
    ax.set_yticks([0, 4, 8])
    ax.set_ylim([0, 8.5])
    ax = ax.twinx() # eof plot
    e_lat.plot(ax=ax, color='black', alpha=.5, linewidth=2)
    # MidLatitudes
    right_eof = e_lat.sel(lat=slice(p9, max_lat))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:green', interpolate=True) # wetter
    left_eof = e_lat.sel(lat=slice(min_lat, p3))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:green', interpolate=True) # wetter
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:orange', interpolate=True) # wet get drier
    if left_eof.mean().data>0: # movement
        pass
    else: # wet get drier
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, facecolor='none', interpolate=True, hatch="xxxx")
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:orange', interpolate=True) # wet get drier
    if right_eof.mean().data>0:
        pass
    else: # wet get drier
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, facecolor='none', interpolate=True, hatch="xxxx")
    # subtropics
    right_eof = e_lat.sel(lat=slice(p7, p9))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:red', interpolate=True) # drier
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:cyan', interpolate=True) # dry get wetter
    if right_eof.mean().data<0: # movement
        pass
    else:
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, facecolor='none', interpolate=True, hatch="xxxx") # dry get wetter
    left_eof = e_lat.sel(lat=slice(p3, p5))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:red', interpolate=True)
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:cyan', interpolate=True)
    if left_eof.mean().data<0:
        pass
    else:
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, facecolor='none', interpolate=True, hatch="xxxx")
    # tropics
    tropic_eof = e_lat.sel(lat=slice(p5, p7))
    ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof>=0, color='tab:green', interpolate=True)
    ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof<=0, color='tab:orange', interpolate=True) # wet get drier
    if tropic_eof.mean().data>0:
        pass
    else:
        ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof<=0, facecolor='none', interpolate=True, hatch='xxxx')
    ax.set_title('')
    ax.set_ylabel('')
    ax.set_ylim([-0.07, 0.07])
    ax.set_yticks([-0.05, 0, 0.05])
    if ind==11 or ind==5:
        pass
    else:
        ax.tick_params(labelright=False)
    ax.axhline(0, color='black', alpha=.5)
plt.tight_layout()
plt.savefig('DDWW-all-MSWEP.png', dpi=180, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(figsize=(14, 7), ncols=6, nrows=4, sharex='row', sharey='row')
for ind, i in enumerate(range(12)):
    # ax = fig.add_subplot(4, 6, ind+1)
    ax = axes.flatten()[ind]
    if ind==0:
        ax.annotate('a)', xy=(-0.15, 1.08), xycoords='axes fraction', weight='bold', fontsize=12)
    p = gpcp_zonal_climate['__xarray_dataarray_variable__'].isel(
        time=i).rolling(lat=5, center=True).mean()
    p = p.dropna(dim='lat')
    min_lat = p.lat.data[0]
    max_lat = p.lat.data[-1]
    wet_indices = argrelextrema(p.data, np.greater)[0]
    e_lat = eofs_all[i].mean(dim='lon').sel(lat=slice(min_lat, max_lat))
    p.plot(ax=ax, linewidth=2, color='tab:blue')
    organized_wet = []
    wet_lat = p.lat[wet_indices].data
    local_maxima_latitudes_sorted = sorted(wet_lat)  # Sort by latitude
    p6 = min(local_maxima_latitudes_sorted, key=lambda x: abs(x))
    organized_wet.append(p6)
    p2 = min([lat for lat in local_maxima_latitudes_sorted if lat < p6], default=None)
    p10 = max([lat for lat in local_maxima_latitudes_sorted if lat > p6], default=None)
    organized_wet.append(p2)
    organized_wet.append(p10)
    organized_dry = []
    left_precip = p.sel(lat=slice(p2, p6))
    p4 = left_precip.lat[left_precip.argmin(dim='lat').data].data
    organized_dry.append(p4)
    right_precip = p.sel(lat=slice(p6, p10))
    p8 = right_precip.lat[right_precip.argmin(dim='lat').data].data
    organized_dry.append(p8)
    p1 = find_closest_lat(p.sel(lat=slice(-90, p2)), p.sel(lat=p2).data / 2)
    p3 = find_closest_lat(p.sel(lat=slice(p2, p4)), (p.sel(lat=p2).data + p.sel(lat=p4).data) / 2)
    p5 = find_closest_lat(p.sel(lat=slice(p4, p6)), (p.sel(lat=p4).data + p.sel(lat=p6).data) / 2)
    p7 = find_closest_lat(p.sel(lat=slice(p6, p8)), (p.sel(lat=p6).data + p.sel(lat=p8).data) / 2)
    p9 = find_closest_lat(p.sel(lat=slice(p8, p10)), (p.sel(lat=p8).data + p.sel(lat=p10).data) / 2)
    p11 = find_closest_lat(p.sel(lat=slice(p10, 90)), p.sel(lat=p10).data / 2)
    ax.set_title(month_names[i])
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.axvspan(p5, p7, alpha=0.15, color='tab:green')
    ax.axvspan(p7, p9, alpha=0.15, color='tab:brown')
    ax.axvspan(p3, p5, alpha=0.15, color='tab:brown')
    ax.axvspan(p9, max_lat, alpha=0.15, color='tab:green')
    ax.axvspan(min_lat, p3, alpha=0.15, color='tab:green')
    ax.set_yticks([0, 2, 4, 6])
    ax.set_ylim([0, 6.5])
    ax = ax.twinx() # eof plot
    e_lat.plot(ax=ax, color='black', alpha=.5, linewidth=2)
    # MidLatitudes
    right_eof = e_lat.sel(lat=slice(p9, max_lat))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:green', interpolate=True) # wetter
    left_eof = e_lat.sel(lat=slice(min_lat, p3))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:green', interpolate=True) # wetter
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:orange', interpolate=True) # wet get drier
    if left_eof.mean().data>0: # movement
        pass
    else: # wet get drier
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, facecolor='none', interpolate=True, hatch="xxxx")
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:orange', interpolate=True) # wet get drier
    if right_eof.mean().data>0:
        pass
    else: # wet get drier
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, facecolor='none', interpolate=True, hatch="xxxx")
    # subtropics
    right_eof = e_lat.sel(lat=slice(p7, p9))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:red', interpolate=True) # drier
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:cyan', interpolate=True) # dry get wetter
    if right_eof.mean().data<0: # movement
        pass
    else:
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, facecolor='none', interpolate=True, hatch="xxxx") # dry get wetter
    left_eof = e_lat.sel(lat=slice(p3, p5))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:red', interpolate=True)
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:cyan', interpolate=True)
    if left_eof.mean().data<0:
        pass
    else:
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, facecolor='none', interpolate=True, hatch="xxxx")
    # tropics
    tropic_eof = e_lat.sel(lat=slice(p5, p7))
    ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof>=0, color='tab:green', interpolate=True)
    ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof<=0, color='tab:orange', interpolate=True) # wet get drier
    if tropic_eof.mean().data>0:
        pass
    else:
        ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof<=0, facecolor='none', interpolate=True, hatch='xxxx')
    ax.set_title('')
    ax.set_ylabel('')
    ax.set_ylim([-0.02, 0.02])
    ax.set_yticks([-0.02, 0, 0.02])
    if ind==11 or ind==5:
        pass
    else:
        ax.tick_params(labelright=False)
    ax.axhline(0, color='black', alpha=.5)
for ind, i in enumerate(range(12)):
    # ax = fig.add_subplot(4, 6, ind+1+12)
    ax = axes.flatten()[ind+12]
    if ind==0:
        ax.annotate('b)', xy=(-0.15, 1.08), xycoords='axes fraction', weight='bold', fontsize=12)
    p = gpcp_zonal_climate_land['precipitation'].isel(
        time=i).rolling(lat=5, center=True).mean()
    p = p.dropna(dim='lat')
    min_lat = p.lat.data[0]
    max_lat = p.lat.data[-1]
    wet_indices = argrelextrema(p.data, np.greater)[0]
    e_lat = eofs_all_land[i].mean(dim='lon').sel(lat=slice(min_lat, max_lat))
    p.plot(ax=ax, linewidth=2, color='tab:blue')
    organized_wet = []
    wet_lat = p.lat[wet_indices].data
    local_maxima_latitudes_sorted = sorted(wet_lat)  # Sort by latitude
    p6 = min(local_maxima_latitudes_sorted, key=lambda x: abs(x))
    organized_wet.append(p6)
    p2 = min([lat for lat in local_maxima_latitudes_sorted if lat < p6], default=min_lat)
    p10 = max([lat for lat in local_maxima_latitudes_sorted if lat > p6], default=max_lat)
    organized_wet.append(p2)
    organized_wet.append(p10)
    organized_dry = []
    left_precip = p.sel(lat=slice(p2, p6))
    p4 = left_precip.lat[left_precip.argmin(dim='lat').data].data
    organized_dry.append(p4)
    right_precip = p.sel(lat=slice(p6, p10))
    p8 = right_precip.lat[right_precip.argmin(dim='lat').data].data
    organized_dry.append(p8)
    p1 = find_closest_lat(p.sel(lat=slice(min_lat, p2)), p.sel(lat=p2).data / 2)
    p3 = find_closest_lat(p.sel(lat=slice(p2, p4)), (p.sel(lat=p2).data + p.sel(lat=p4).data) / 2)
    p5 = find_closest_lat(p.sel(lat=slice(p4, p6)), (p.sel(lat=p4).data + p.sel(lat=p6).data) / 2)
    p7 = find_closest_lat(p.sel(lat=slice(p6, p8)), (p.sel(lat=p6).data + p.sel(lat=p8).data) / 2)
    p9 = find_closest_lat(p.sel(lat=slice(p8, p10)), (p.sel(lat=p8).data + p.sel(lat=p10).data) / 2)
    p11 = find_closest_lat(p.sel(lat=slice(p10, max_lat)), p.sel(lat=p10).data / 2)
    ax.set_title(month_names[i])
    ax.set_ylabel('')
    ax.set_xlabel('')
    ax.axvspan(p5, p7, alpha=0.15, color='tab:green')
    ax.axvspan(p7, p9, alpha=0.15, color='tab:brown')
    ax.axvspan(p3, p5, alpha=0.15, color='tab:brown')
    ax.axvspan(p9, max_lat, alpha=0.15, color='tab:green')
    ax.axvspan(min_lat, p3, alpha=0.15, color='tab:green')
    ax.set_yticks([0, 4, 8])
    ax.set_ylim([0, 8.5])
    ax = ax.twinx() # eof plot
    e_lat.plot(ax=ax, color='black', alpha=.5, linewidth=2)
    # MidLatitudes
    right_eof = e_lat.sel(lat=slice(p9, max_lat))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:green', interpolate=True) # wetter
    left_eof = e_lat.sel(lat=slice(min_lat, p3))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:green', interpolate=True) # wetter
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:orange', interpolate=True) # wet get drier
    if left_eof.mean().data>0: # movement
        pass
    else: # wet get drier
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, facecolor='none', interpolate=True, hatch="xxxx")
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:orange', interpolate=True) # wet get drier
    if right_eof.mean().data>0:
        pass
    else: # wet get drier
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, facecolor='none', interpolate=True, hatch="xxxx")
    # subtropics
    right_eof = e_lat.sel(lat=slice(p7, p9))
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof<=0, color='tab:red', interpolate=True) # drier
    ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, color='tab:cyan', interpolate=True) # dry get wetter
    if right_eof.mean().data<0: # movement
        pass
    else:
        ax.fill_between(right_eof.lat.data, right_eof.data, 0, alpha=.5, where=right_eof>=0, facecolor='none', interpolate=True, hatch="xxxx") # dry get wetter
    left_eof = e_lat.sel(lat=slice(p3, p5))
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof<=0, color='tab:red', interpolate=True)
    ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, color='tab:cyan', interpolate=True)
    if left_eof.mean().data<0:
        pass
    else:
        ax.fill_between(left_eof.lat.data, left_eof.data, 0, alpha=.5, where=left_eof>=0, facecolor='none', interpolate=True, hatch="xxxx")
    # tropics
    tropic_eof = e_lat.sel(lat=slice(p5, p7))
    ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof>=0, color='tab:green', interpolate=True)
    ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof<=0, color='tab:orange', interpolate=True) # wet get drier
    if tropic_eof.mean().data>0:
        pass
    else:
        ax.fill_between(tropic_eof.lat.data, tropic_eof.data, 0, alpha=.5, where=tropic_eof<=0, facecolor='none', interpolate=True, hatch='xxxx')
    ax.set_title('')
    ax.set_ylabel('')
    ax.set_ylim([-0.07, 0.07])
    ax.set_yticks([-0.05, 0, 0.05])
    if ind==11 or ind==5:
        pass
    else:
        ax.tick_params(labelright=False)
    ax.axhline(0, color='black', alpha=.5)
plt.tight_layout()
plt.savefig('DDWW-all-gpcp.png', dpi=180, bbox_inches='tight')
plt.show()